In [40]:
# 
abstract type AbstractIndex end

struct Index <: AbstractIndex
    lookup::Dict{Symbol, Int}
    names::Vector{Symbol}
end

function Index(names::AbstractVector{Symbol}; makeunique::Bool=false)
    u = make_unique(names, makeunique=makeunique)
    lookup = Dict{Symbol, Int}(zip(u, 1:length(u)))
    return Index(lookup, u)
end

Index() = Index(Dict{Symbol, Int}(), Symbol[])
Base.length(x::Index) = length(x.names)
Base.names(x::Index) = string.(x.names)

In [43]:
abstract type AbstractDataFrame end
struct DataFrame <: AbstractDataFrame
    columns::Vector{AbstractVector}
    colindex::Index
    
    #内部构造器
    function DataFrame(columns::Union{Vector{Any}, Vector{AbstractVector}},
        colindex::Index; copycols::Bool=true)
        if length(columns) == length(colindex) == 0
            return new(AbstractVector[], Index())
        elseif length(columns) != length(colindex)
            throw(DimensionMismatch("Number of columns ($(length(columns))) and number of" *
                    " column names($(length(colindex)) are not equal"))
        end
        
        len = -1
        firstvec = -1
        for (i, col) in enumerate(columns)
            if col isa AbstractVector
                if len == -1
                    len = length(col)
                    firstvec = i
                elseif len != length(col)
                    n1 = _names(colindex)[firstvec]
                    n2 = _names(colindex)[i]
                    throw(DimensionMismatch("column :$n1 has length $len and column " *
                                            ":$n2 has length $(length(col))"))
                end
            end
        end
        len == -1 && (len = 1)

        # we write into columns as we know that it is guaranteed
        # that it was freshly allocated in the outer constructor
        for (i, col) in enumerate(columns)
            # check for vectors first as they are most common
            if col isa AbstractRange
                columns[i] = collect(col)
            elseif col isa AbstractVector
                columns[i] = copycols ? copy(col) : col
            elseif col isa Union{AbstractArray{<:Any, 0}, Ref}
                x = col[]
                columns[i] = fill!(Tables.allocatecolumn(typeof(x), len), x)
            else
                if col isa AbstractArray
                    throw(ArgumentError("adding AbstractArray other than AbstractVector" *
                                        " as a column of a data frame is not allowed"))
                end
                columns[i] = fill!(Tables.allocatecolumn(typeof(col), len), col)
            end
        end

        new(convert(Vector{AbstractVector}, columns), colindex)
    end
end

function DataFrame(; kwargs...)
    if isempty(kwargs)
        DataFrame([], Index())
    else
        cnames = Symbol[]
        columns = Any[]
        copycols = true
        for (kw, val) in kwargs
            if kw == :copycols
                if val isa Bool
                    copycols = val
                else
                    throw(ArgumentError("the `copycols` keyword argument must be Boolean"))
                end
            else
                push!(cnames, kw)
                push!(columns, val)
            end
        end
        DataFrame(columns, Index(cnames), copycols=copycols)
    end
end



DataFrame

In [44]:
# 测试用格子
DataFrame()

DataFrame(AbstractArray{T,1} where T[], Index(Dict{Symbol,Int64}(), Symbol[]))

In [45]:
using Random
DataFrame(A=1:3, B = rand(3), C = randstring.([3,3,3]), fixed=1)

UndefVarError: UndefVarError: make_unique not defined

In [38]:
true && (len = -1)

-1

In [39]:
len

-1